## Análise comparativa dos dados

In [1]:
# pip install -U scikit-learn

In [3]:
import pandas as pd
from IPython.display import display, Markdown, HTML
import joblib
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

### 01. Importação dos dados

In [4]:
df = pd.read_csv("../data/raw/data.csv")
dict = pd.read_csv("../data/external/dictionary.csv")

display(df.describe(include='all'))
display(dict)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
count,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,889,891,891,891,203,889,891,891
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,7,3,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,C,Southampton,no,True
freq,NaN,NaN,577,NaN,NaN,NaN,NaN,644,491,537,537,59,644,549,537
mean,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.486592,0.836071,NaN,14.526497,1.102743,0.806057,49.693429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,2.000000,NaN,20.125000,0.000000,0.000000,7.910400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,3.000000,NaN,28.000000,0.000000,0.000000,14.454200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,0.000000,31.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,variavel,descrição,tipo,subtipo
0,survived,Se a pessoa sobreviveu ou não,qualitativo,nominal
1,pclass,"CLasse da passagem (1,2,3)",qualitativo,ordinal
2,sex,Gênero do passageiro,qualitativo,nominal
3,age,Idade do passageiro,quantitativo,contínua
4,sibsp,Se o passageiro possui irmãos/cônjuges a bordo,quantitativo,discreto
5,parch,Se o passageiro possui pais/filhos a bordo,quantitativo,discreto
6,fare,Valor da passagem,quantitativo,contínuo
7,embarked,"Porto de embarque (C, Q, S)",qualitativo,nominal
8,class,"Classe do passageiro (1,2,3)",qualitativo,ordinal
9,who,Perfil do passageiro,qualitativo,nominal


In [5]:
# Observando conjunto amostral
uniques = {'Variavel': [], 'Tipo': [], 'Valores': []}

for variable in dict['variavel']:
    if variable in df.columns:
        uniques['Variavel'].append(variable)
        uniques['Tipo'].append(df[variable].dtype)
        uniques['Valores'].append(set(df[variable].unique()))
    else:
        uniques['Variavel'].append(variable)
        uniques['Tipo'].append(None)
        uniques['Valores'].append(set())

df_uniques = pd.DataFrame.from_dict(uniques)
display(HTML(df_uniques.to_html()))

,Variavel,Tipo,Valores
0,survived,int64,"{0, 1}"
1,pclass,int64,"{1, 2, 3}"
2,sex,object,"{male, female}"
3,age,float64,"{0.75, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 28.5, 31.0, 32.0, 30.0, 34.0, 35.0, 33.0, 37.0, 38.0, 39.0, 40.0, 32.5, 42.0, 36.5, 40.5, 45.0, 46.0, 47.0, 44.0, 49.0, 50.0, 51.0, 45.5, 52.0, 54.0, 55.0, 55.5, 56.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 70.5, 71.0, 14.5, 70.0, 74.0, 80.0, 20.5, 23.5, 24.5, 0.42, 30.5, 34.5, 36.0, 41.0, 43.0, 48.0, 0.83, 0.92, 0.67, nan, 53.0, 57.0}"
4,sibsp,int64,"{0, 1, 2, 3, 4, 5, 8}"
5,parch,int64,"{0, 1, 2, 3, 4, 5, 6}"
6,fare,float64,"{0.0, 512.3292, 4.0125, 5.0, 6.975, 7.925, 8.05, 7.25, 8.4583, 11.1333, 7.8542, 13.0, 7.225, 8.0292, 16.7, 16.0, 18.0, 11.2417, 15.5, 21.075, 21.0, 21.6792, 17.8, 20.575, 26.55, 26.0, 27.7208, 29.125, 30.0708, 31.275, 31.3875, 27.75, 27.9, 35.5, 7.75, 29.0, 34.375, 39.6875, 34.6542, 41.5792, 7.1417, 36.75, 7.7958, 7.3125, 46.9, 9.0, 9.5, 47.1, 50.0, 51.8625, 10.5, 53.1, 52.0, 55.0, 56.4958, 11.5, 52.5542, 56.9292, 57.9792, 61.9792, 61.175, 63.3583, 61.3792, 12.875, 66.6, 13.5, 12.0, 69.55, 69.3, 71.2833, 14.4542, 73.5, 14.5, 14.0, 76.7292, 77.2875, 15.0458, 79.2, 80.0, 16.1, 82.1708, 83.475, 15.75, 76.2917, 86.5, 79.65, 83.1583, 81.8583, 90.0, 18.7875, 91.0792, 18.75, 89.1042, 93.5, 19.5, 19.2583, 7.2292, 20.525, 20.2125, 20.25, ...}"
7,embarked,object,"{nan, S, C, Q}"
8,class,object,"{First, Third, Second}"
9,who,object,"{child, man, woman}"


,Variavel,Valores
0,survived,"[0, 1]"
1,pclass,"[3, 1, 2]"
2,sex,"[male, female]"
3,age,"[22.0, 38.0, 26.0, 35.0, nan, 54.0, 2.0, 27.0, 14.0, 4.0, 58.0, 20.0, 39.0, 55.0, 31.0, 34.0, 15.0, 28.0, 8.0, 19.0, 40.0, 66.0, 42.0, 21.0, 18.0, 3.0, 7.0, 49.0, 29.0, 65.0, 28.5, 5.0, 11.0, 45.0, 17.0, 32.0, 16.0, 25.0, 0.83, 30.0, 33.0, 23.0, 24.0, 46.0, 59.0, 71.0, 37.0, 47.0, 14.5, 70.5, 32.5, 12.0, 9.0, 36.5, 51.0, 55.5, 40.5, 44.0, 1.0, 61.0, 56.0, 50.0, 36.0, 45.5, 20.5, 62.0, 41.0, 52.0, 63.0, 23.5, 0.92, 43.0, 60.0, 10.0, 64.0, 13.0, 48.0, 0.75, 53.0, 57.0, 80.0, 70.0, 24.5, 6.0, 0.67, 30.5, 0.42, 34.5, 74.0]"
4,sibsp,"[1, 0, 3, 4, 2, 5, 8]"
5,parch,"[0, 1, 2, 5, 3, 4, 6]"
6,fare,"[7.25, 71.2833, 7.925, 53.1, 8.05, 8.4583, 51.8625, 21.075, 11.1333, 30.0708, 16.7, 26.55, 31.275, 7.8542, 16.0, 29.125, 13.0, 18.0, 7.225, 26.0, 8.0292, 35.5, 31.3875, 263.0, 7.8792, 7.8958, 27.7208, 146.5208, 7.75, 10.5, 82.1708, 52.0, 7.2292, 11.2417, 9.475, 21.0, 41.5792, 15.5, 21.6792, 17.8, 39.6875, 7.8, 76.7292, 61.9792, 27.75, 46.9, 80.0, 83.475, 27.9, 15.2458, 8.1583, 8.6625, 73.5, 14.4542, 56.4958, 7.65, 29.0, 12.475, 9.0, 9.5, 7.7875, 47.1, 15.85, 34.375, 61.175, 20.575, 34.6542, 63.3583, 23.0, 77.2875, 8.6542, 7.775, 24.15, 9.825, 14.4583, 247.5208, 7.1417, 22.3583, 6.975, 7.05, 14.5, 15.0458, 26.2833, 9.2167, 79.2, 6.75, 11.5, 36.75, 7.7958, 12.525, 66.6, 7.3125, 61.3792, 7.7333, 69.55, 16.1, 15.75, 20.525, 55.0, 25.925, ...]"
7,embarked,"[S, C, Q, nan]"
8,class,"[Third, First, Second]"
9,who,"[man, woman, child]"


### 02. Preparação dos Dados

In [ ]:
target_column = 'who'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [ ]:
# tratamento de dados discrepantes
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

model = LogisticRegression()

#### Observando valores únicos:

### Tratamento de dados faltantes: 

- identificar e remover linhas/colunas com dados faltantes insignificatnes; imputar valores ausentes usando estatísticas de tendência central ou modelos preditivos.


### Tratamento de dados discrepantes: 

- remover ou isolar outliers dependendo do contexto.